### 查看GPU

In [2]:
!nvidia-smi

Thu Apr 18 14:15:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 安裝需要的BERT套件

In [3]:
!pip install transformers==4.29.0 -U
!pip install nlp -U
!pip install torch==2.0.0 -U
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 3.1 MB/s eta 0:00:00
    

### 載入套件

In [4]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset, Dataset
import nlp
import torch
import random
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### 抓取訓練資料

In [5]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv

--2024-04-18 14:27:33--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-18 14:27:33--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.03s   

2024-04-18 14:27:34 (10.7 MB/s) - ‘train.csv’ saved [388895/388895]



### 讀取訓練資料集

In [6]:
df = pd.read_csv('train.csv')
dataset = Dataset.from_pandas(df)

In [7]:
df.sample(10)

,ID,Name,Description,label
887,887,https://tw.buy.yahoo.com/gdsale/GNC健安喜-專注力up-L...,含有白果、魚油、磷脂醯絲胺酸、五味子及L-肉酸等多種的配方。適用對象：壓力大的上班族、常思考...,0
521,521,黃清香即蘑菇大師專賣店,「美麗不挨餓健康美出來」「硒可以預防腫瘤癌症…那些服用硒補充物的第一組患者，胃癌的機率…少五...,1
650,650,愛尚它?葡萄籽、松樹皮、紅酒萃取精華粉末,愛尚它?產品係經過科學研擬，以果糖、D-葡萄糖、檸檬酸、碳酸氫鉀和其它成分精心配製而成，當您...,0
449,449,長庚團隊研發SOD活酵素飲,「提升人體抵抗力200%...並促進細胞新生100倍...讓肌膚白裡透紅...可以使受損細胞...,1
926,926,https://tw.buy.yahoo.com/gdsale/GNC健安喜-三效維生素C5...,新加坡銷售第一名的維他命C。-為一般維生素C的三倍功效。-為水溶性、PH值接近中性的維他命C...,0
850,850,https://tw.buy.yahoo.com/gdsale/GNC健安喜-鈣有骨氣-維他...,【維他命D】GNC維生素D3(人體最能吸收的型式)，方便達到每日建議攝取的維生素D含量。愈來...,0
115,115,「怡日健粉妍片、酵母葡聚糖粉末」,「維他命C是最棒的解毒劑、維他命E-幫助膠原蛋白生成..EGCG：使腫瘤縮小，抗癌第一..抵...,1
454,454,ST.MALO斯里蘭卡冷壓初榨椰子油-500ML+200ML,「生飲-每日10ML可補充營養及潤腸排毒…育兒-有利於傷口殺菌舒緩皮膚濕疹和蚊蟲叮癢」,1
159,159,海之滴褐藻糖膠,「…重症治療…慢性疾病或重症患者…接受化學藥物治療的重症病人亦可同時服用海之滴褐藻糖膠…適合...,1
852,852,https://tw.buy.yahoo.com/gdsale/GNC健安喜-超值3入組-L...,●100%頂級印尼金絲燕燕盞●低熱量、低糖，每份熱量僅有8.5kcal●熱穿透滅菌技術、真空...,0


### 載入原始模型

In [8]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

###  載入分詞模型

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

### 設定執行參數

In [10]:
RANDOM_SEED = 5
MAX_LEN = 512
EPOCHS=5
BATCH_SIZE = 8

### 把資料集分成訓練、測試、驗證

In [11]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 取一筆出來瞧瞧(已經打亂)

In [12]:
train_dataset[2]

{'Description': '「從根源打擊游離脂肪…快、狠、準，立即分解減吋…進而讓身體成為易瘦體質…經臨床功效驗證…受試者引用24小時後脂肪立即分解了213％…持續飲用14天…體重明顯地降低了4.8…減脂」',
 'ID': 465,
 'Name': '伊森生醫名模完美曲線美纖飲',
 'label': 1}

In [13]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

### 定義分詞器

In [14]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

### 定義評測準則，訓練器

In [15]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=50,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

### 開始訓練

In [16]:
trainer.train()
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.00029545516008511186,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_runtime': 2.6022,
 'eval_samples_per_second': 36.123,
 'eval_steps_per_second': 4.611,
 'epoch': 5.0}

### 找一個範例來測試

In [17]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])
print(df.iloc[578]['label'], df.iloc[578]['Description'])

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


### 設定測試文字

In [18]:
#test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'
test_text = '???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品'

MAX_LEN = 512

encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
)

In [19]:
encoded_review

{'input_ids': tensor([[  101,   136,   136,   136,   136,   136,   136,   136,  3680,   819,
          1419,  3300,  8114,  8882,  6737, 11382,   511,  6737, 11382,  1377,
          1347,  5645,  5204,  4495,  5162,   147,  1086,  4495,  8024,  1060,
          5442,  1072,  1295,  1398,   868,  4500,  8024,  1377,   914,  6868,
          3173,  7376,   807,  6342,   511,   924,  6362,  2542,  4472,  7518,
          3268,  5204,  2898,   978,  2434,   511,  3268,  7077,  3189,  3315,
          4638,  3297,   881,  5401,  2159,  5469,  1501,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

### 進行測試

In [20]:
test_model = model.to('cuda')

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
output = test_model(input_ids, attention_mask)
result = torch.argmax(output[0][0])

classnames = ['合格', '不合格']

print(f'廣告詞: {test_text}')
print(f'檢測結果: {classnames[result]}')

廣告詞: ???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品
檢測結果: 合格
